In [14]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Step 1: Remove 'is_mobile' column
train_df.drop('is_mobile', axis=1, inplace=True)
test_df.drop('is_mobile', axis=1, inplace=True)

# Display the first few rows of the modified dataframes
train_df.head(), test_df.head()


(   index  gender  is_car  is_reality  income_year           income_type  \
 0      0       1       0           0     202500.0  Commercial associate   
 1      1       1       0           1     247500.0  Commercial associate   
 2      2       0       1           1     450000.0               Working   
 3      3       1       0           1     202500.0  Commercial associate   
 4      4       1       1           1     157500.0         State servant   
 
                         edu_type  marital status           house_type  \
 0               Higher education         Married  Municipal apartment   
 1  Secondary / secondary special  Civil marriage    House / apartment   
 2               Higher education         Married    House / apartment   
 3  Secondary / secondary special         Married    House / apartment   
 4               Higher education         Married    House / apartment   
 
    is_email     job_type  family_size  credit  
 0         0          NaN            2       1 

In [15]:
# Describe the 'family_size' column to understand its distribution
train_family_size_desc = train_df['family_size'].describe()

# Calculate IQR for 'family_size' to identify outliers
Q1 = train_df['family_size'].quantile(0.25)
Q3 = train_df['family_size'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Count how many outliers there are
outliers_count = train_df[(train_df['family_size'] < lower_bound) | (train_df['family_size'] > upper_bound)].shape[0]

train_family_size_desc, outliers_count


(count    22001.000000
 mean         2.198809
 std          0.915285
 min          1.000000
 25%          2.000000
 50%          2.000000
 75%          3.000000
 max         20.000000
 Name: family_size, dtype: float64,
 298)

In [16]:
# Remove outliers in the train dataset
train_df_no_outliers = train_df[(train_df['family_size'] >= lower_bound) & (train_df['family_size'] <= upper_bound)]

# Display the shape of the dataframe before and after outlier removal
original_shape = train_df.shape
new_shape = train_df_no_outliers.shape

original_shape, new_shape


((22001, 13), (21703, 13))

In [17]:
from sklearn.preprocessing import LabelEncoder

# Fill missing values for 'job_type' with 'unknown'
train_df_no_outliers['job_type'].fillna('unknown', inplace=True)
test_df['job_type'].fillna('unknown', inplace=True)

# Initialize LabelEncoder
le = LabelEncoder()

# Apply Label Encoding to categorical columns with object datatype
categorical_cols = train_df_no_outliers.select_dtypes(include=['object']).columns

# Apply the label encoder to each categorical column
for col in categorical_cols:
    train_df_no_outliers[col] = le.fit_transform(train_df_no_outliers[col])
    # We fit the encoder on the train data and transform both train and test data
    # This ensures that the same encoding is applied to both
    test_df[col] = le.transform(test_df[col])

# Display the first few rows of the modified dataframes to confirm changes
train_df_no_outliers.head(), test_df.head()


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/tmp/ipykernel_146555/1326949545.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_no_outliers[col] = le.fit_transform(train_df_no_outliers[col])
/tmp/ipykernel_146555/1326949545.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

(   index  gender  is_car  is_reality  income_year  income_type  edu_type  \
 0      0       1       0           0     202500.0            0         1   
 1      1       1       0           1     247500.0            0         4   
 2      2       0       1           1     450000.0            4         1   
 3      3       1       0           1     202500.0            0         4   
 4      4       1       1           1     157500.0            2         1   
 
    marital status  house_type  is_email  job_type  family_size  credit  
 0               1           2         0        18            2       1  
 1               0           1         1         8            3       1  
 2               1           1         0        10            2       2  
 3               1           1         0        14            2       0  
 4               1           1         0        10            2       2  ,
    index  gender  is_car  is_reality  income_year  income_type  edu_type  \
 0  22001     

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the data into features and target
X = train_df_no_outliers.drop('credit', axis=1)  # drop the target variable to isolate features
y = train_df_no_outliers['credit']  # target variable

# Split the train data into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the RandomForestClassifier
rf_classifier.fit(X_train, y_train)

# Predict on the validation set
y_pred_val = rf_classifier.predict(X_val)

# Calculate accuracy on the validation set
accuracy = accuracy_score(y_val, y_pred_val)

# Output the accuracy
accuracy


0.631882054826077

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_moons
# Define a grid of hyperparameters to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required at each leaf node
    'bootstrap': [True, False]        # Method of selecting samples for training each tree
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,  # Number of folds in cross-validation
                           n_jobs=-1,  # Number of jobs to run in parallel (-1 will use all processors)
                           verbose=2)  # Controls the verbosity of the job

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score achieved during the GridSearch
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.3s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   7.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; tot

[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   3.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   5.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   5.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   8.4s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   5.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   2.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   1.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   5.8s
[CV] END bootstrap=True, max_depth=None, min_s

[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.9s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.3s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.3s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.3s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   3.9s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.3s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   3.5s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_s

[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   3.4s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   1.2s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   1.2s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   1.4s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   1.8s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   1.8s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   5.5s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   3.7s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2,

[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.0s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.0s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   6.1s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   9.4s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   9.5s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.9s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   6.0s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.9s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=1, min_s

...

[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=  12.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   7.9s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   2.4s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   2.5s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   7.8s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   2.5s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   2.5s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   2.4s
[CV] END bootstrap=False, max_depth=20, min_samples

[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   4.6s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   7.4s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   4.5s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   4.5s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   6.9s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   7.0s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=  11.5s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   6.9s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf

[CV] END bootstrap=False, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   2.7s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   2.8s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   2.7s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   4.2s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   6.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   5.4s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   5.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   9.0s
[CV] END bootstrap=False, max_depth=30, min_samp

[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   7.4s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=  11.4s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   7.1s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   7.2s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   2.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   3.8s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   2.5s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   2.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf

({'bootstrap': False,
  'max_depth': 20,
  'min_samples_leaf': 1,
  'min_samples_split': 10,
  'n_estimators': 300},
 0.6548785993596421)

In [22]:
# Initialize the RandomForestClassifier with the best parameters
rf_classifier_optimized = RandomForestClassifier(
    bootstrap=False,
    max_depth=20,
    min_samples_leaf=1,
    min_samples_split=10,
    n_estimators=300,
    random_state=42
)

# Train the RandomForestClassifier
rf_classifier_optimized.fit(X_train, y_train)

# Predict on the validation set
y_pred_val_optimized = rf_classifier_optimized.predict(X_val)

# Calculate accuracy on the validation set
accuracy_optimized = accuracy_score(y_val, y_pred_val_optimized)

# Output the accuracy
accuracy_optimized


0.6731167933655839

In [23]:
# Initialize the RandomForestClassifier with the optimized hyperparameters
rf_classifier_final = RandomForestClassifier(
    bootstrap=False,
    max_depth=20,
    min_samples_leaf=1,
    min_samples_split=10,
    n_estimators=300,
    random_state=42
)

# Train the RandomForestClassifier on the entire training dataset (without outliers)
rf_classifier_final.fit(train_df_no_outliers.drop('credit', axis=1), train_df_no_outliers['credit'])

# Predict on the test dataset
test_predictions = rf_classifier_final.predict(test_df)

# Add predictions to the test dataframe
test_df['credit'] = test_predictions

# Save the test dataframe with predictions to a new CSV file
prediction_file_path = 'test_predictions.csv'
test_df.to_csv(prediction_file_path, index=False)

prediction_file_path



'test_predictions.csv'